In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("./Resources/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(['EIN','NAME'],1)
application_df.head()

/var/folders/tk/zhnlhwsd0w734ckw7_z84vb80000gn/T/ipykernel_2420/4242059896.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  application_df = application_df.drop(['EIN','NAME'],1)


,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [3]:

# Determine the number of unique values in each column.
application_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [4]:
# Look at APPLICATION_TYPE value counts for binning
application_count = application_df.APPLICATION_TYPE.value_counts()
application_count

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [5]:

# Determine which values to replace if counts are less than ...?
replace_application = list(application_count[application_count < 500].index)

# Replace in dataframe
for app in replace_application:
    application_df.APPLICATION_TYPE = application_df.APPLICATION_TYPE.replace(app,"Other")
    
# Check to make sure binning was successful
application_df.APPLICATION_TYPE.value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [6]:
# Look at CLASSIFICATION value counts for binning
classification_count = application_df.CLASSIFICATION.value_counts()
classification_count

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [7]:

# Determine which values to replace if counts are less than ..?
replace_class = list(classification_count[classification_count < 1800].index)

# Replace in dataframe
for cls in replace_class:
    application_df.CLASSIFICATION = application_df.CLASSIFICATION.replace(cls,"Other")
    
# Check to make sure binning was successful
application_df.CLASSIFICATION.value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [8]:
income_count = application_df.INCOME_AMT.value_counts()
income_count

0                24388
25000-99999       3747
100000-499999     3374
1M-5M              955
1-9999             728
10000-24999        543
10M-50M            240
5M-10M             185
50M+               139
Name: INCOME_AMT, dtype: int64

In [9]:
# Determine which values to replace if counts are less than ..?
replace_income = list(income_count[income_count < 300].index)

# Replace in dataframe
for cls in replace_income:
    application_df.INCOME_AMT = application_df.INCOME_AMT.replace(cls,"Other")
    
# Check to make sure binning was successful
application_df.INCOME_AMT.value_counts()

0                24388
25000-99999       3747
100000-499999     3374
1M-5M              955
1-9999             728
Other              564
10000-24999        543
Name: INCOME_AMT, dtype: int64

In [10]:
# Generate our categorical variable lists
application_cat = application_df.dtypes[application_df.dtypes == 'object'].index.tolist()
application_df[application_cat].nunique()

APPLICATION_TYPE          9
AFFILIATION               6
CLASSIFICATION            6
USE_CASE                  5
ORGANIZATION              4
INCOME_AMT                7
SPECIAL_CONSIDERATIONS    2
dtype: int64

In [11]:

# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(application_df[application_cat]))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names(application_cat)
encode_df.head()

,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,APPLICATION_TYPE_T8,AFFILIATION_CompanySponsored,...,ORGANIZATION_Trust,INCOME_AMT_0,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_Other,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0


In [12]:
# Merge one-hot encoded features and drop the originals
application_df = application_df.merge(encode_df, left_index=True, right_index=True)
application_df = application_df.drop(application_cat,1)
application_df.head()

/var/folders/tk/zhnlhwsd0w734ckw7_z84vb80000gn/T/ipykernel_2420/108820884.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  application_df = application_df.drop(application_cat,1)


,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,ORGANIZATION_Trust,INCOME_AMT_0,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_Other,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,1,108590,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,1,5000,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,1,6692,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
4,1,142590,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0


In [13]:

# Split our preprocessed data into our features and target arrays
y = application_df['IS_SUCCESSFUL'].values
X = application_df.drop('IS_SUCCESSFUL',1).values

# Split the preprocessed data into a training and testing dataset
X_train,X_test,y_train,y_test = train_test_split(X,y,random_state=42, stratify=y)

/var/folders/tk/zhnlhwsd0w734ckw7_z84vb80000gn/T/ipykernel_2420/1945616859.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  X = application_df.drop('IS_SUCCESSFUL',1).values


In [14]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [15]:

# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.

number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 = 80
hidden_nodes_layer2 = 30

nn = Sequential()

# First hidden layer
nn.add(Dense(units=hidden_nodes_layer1,input_dim=number_input_features,activation='relu'))

# Second hidden layer
nn.add(Dense(units=hidden_nodes_layer2,activation='relu'))

# Output layer
nn.add(Dense(units=1,activation='sigmoid'))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 80)                3360      
_________________________________________________________________
dense_1 (Dense)              (None, 30)                2430      
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 31        
Total params: 5,821
Trainable params: 5,821
Non-trainable params: 0
_________________________________________________________________


2021-10-24 17:01:35.372526: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [16]:
# Compile the model
nn.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [17]:
# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=100)

2021-10-24 17:01:51.245465: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/100
804/804 [==============================] - 1s 616us/step - loss: 0.5689 - accuracy: 0.7226
Epoch 2/100
804/804 [==============================] - 1s 637us/step - loss: 0.5572 - accuracy: 0.7315
Epoch 3/100
804/804 [==============================] - 0s 594us/step - loss: 0.5528 - accuracy: 0.7310
Epoch 4/100
804/804 [==============================] - 1s 625us/step - loss: 0.5515 - accuracy: 0.7318
Epoch 5/100
804/804 [==============================] - 1s 922us/step - loss: 0.5490 - accuracy: 0.7320
Epoch 6/100
804/804 [==============================] - 1s 789us/step - loss: 0.5486 - accuracy: 0.7350
Epoch 7/100
804/804 [==============================] - 1s 749us/step - loss: 0.5485 - accuracy: 0.7329
Epoch 8/100
804/804 [==============================] - 0s 620us/step - loss: 0.5474 - accuracy: 0.7338
Epoch 9/100
804/804 [==============================] - 1s 681us/step - loss: 0.5470 - accuracy: 0.7347
Epoch 10/100
804/804 [==============================] - 1s 700us/step - l

804/804 [==============================] - 1s 651us/step - loss: 0.5359 - accuracy: 0.7406
Epoch 80/100
804/804 [==============================] - 1s 706us/step - loss: 0.5359 - accuracy: 0.7404
Epoch 81/100
804/804 [==============================] - 1s 871us/step - loss: 0.5361 - accuracy: 0.7407
Epoch 82/100
804/804 [==============================] - 1s 792us/step - loss: 0.5358 - accuracy: 0.7409
Epoch 83/100
804/804 [==============================] - 1s 644us/step - loss: 0.5357 - accuracy: 0.7403
Epoch 84/100
804/804 [==============================] - 1s 632us/step - loss: 0.5358 - accuracy: 0.7407
Epoch 85/100
804/804 [==============================] - 1s 644us/step - loss: 0.5355 - accuracy: 0.7406
Epoch 86/100
804/804 [==============================] - 1s 690us/step - loss: 0.5355 - accuracy: 0.7402
Epoch 87/100
804/804 [==============================] - 1s 678us/step - loss: 0.5360 - accuracy: 0.7411
Epoch 88/100
804/804 [==============================] - 1s 648us/step - loss:

In [21]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5699 - accuracy: 0.7248
Loss: 0.5698808431625366, Accuracy: 0.724781334400177


In [19]:
# Import checkpoint dependencies
import os
from tensorflow.keras.callbacks import ModelCheckpoint

# Define the checkpoint path and filenames
os.makedirs('checkpoints/Optimization1/',exist_ok=True)
checkpoint_path = 'checkpoints/Optimization1/weights.{epoch:02d}.hdf5'

# Create a callback that saves the model's weights every 5 epochs
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq=100)

# Compile the model
nn.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=100,callbacks=[cp_callback])

Epoch 1/100
 69/804 [=>............................] - ETA: 0s - loss: 0.5162 - accuracy: 0.7518  
Epoch 00001: saving model to checkpoints/Optimization1/weights.01.hdf5
134/804 [====>.........................] - ETA: 0s - loss: 0.5269 - accuracy: 0.7428
Epoch 00001: saving model to checkpoints/Optimization1/weights.01.hdf5
270/804 [=========>....................] - ETA: 0s - loss: 0.5352 - accuracy: 0.7383
Epoch 00001: saving model to checkpoints/Optimization1/weights.01.hdf5
339/804 [===========>..................] - ETA: 0s - loss: 0.5397 - accuracy: 0.7358
Epoch 00001: saving model to checkpoints/Optimization1/weights.01.hdf5
474/804 [================>.............] - ETA: 0s - loss: 0.5379 - accuracy: 0.7388
Epoch 00001: saving model to checkpoints/Optimization1/weights.01.hdf5
548/804 [===================>..........] - ETA: 0s - loss: 0.5353 - accuracy: 0.7410
Epoch 00001: saving model to checkpoints/Optimization1/weights.01.hdf5
681/804 [========================>.....] - ETA: 0s

 59/804 [=>............................] - ETA: 0s - loss: 0.5315 - accuracy: 0.7436
Epoch 00007: saving model to checkpoints/Optimization1/weights.07.hdf5
172/804 [=====>........................] - ETA: 0s - loss: 0.5310 - accuracy: 0.7476
Epoch 00007: saving model to checkpoints/Optimization1/weights.07.hdf5
245/804 [========>.....................] - ETA: 0s - loss: 0.5328 - accuracy: 0.7458
Epoch 00007: saving model to checkpoints/Optimization1/weights.07.hdf5
319/804 [==========>...................] - ETA: 0s - loss: 0.5343 - accuracy: 0.7452
Epoch 00007: saving model to checkpoints/Optimization1/weights.07.hdf5
395/804 [=============>................] - ETA: 0s - loss: 0.5337 - accuracy: 0.7441
Epoch 00007: saving model to checkpoints/Optimization1/weights.07.hdf5
545/804 [===================>..........] - ETA: 0s - loss: 0.5312 - accuracy: 0.7446
Epoch 00007: saving model to checkpoints/Optimization1/weights.07.hdf5
636/804 [======================>.......] - ETA: 0s - loss: 0.533

  1/804 [..............................] - ETA: 1s - loss: 0.5360 - accuracy: 0.7188
Epoch 00013: saving model to checkpoints/Optimization1/weights.13.hdf5
121/804 [===>..........................] - ETA: 0s - loss: 0.5356 - accuracy: 0.7397
Epoch 00013: saving model to checkpoints/Optimization1/weights.13.hdf5
228/804 [=======>......................] - ETA: 0s - loss: 0.5307 - accuracy: 0.7460
Epoch 00013: saving model to checkpoints/Optimization1/weights.13.hdf5
339/804 [===========>..................] - ETA: 0s - loss: 0.5313 - accuracy: 0.7455
Epoch 00013: saving model to checkpoints/Optimization1/weights.13.hdf5
408/804 [==============>...............] - ETA: 0s - loss: 0.5347 - accuracy: 0.7435
Epoch 00013: saving model to checkpoints/Optimization1/weights.13.hdf5
511/804 [==================>...........] - ETA: 0s - loss: 0.5329 - accuracy: 0.7432
Epoch 00013: saving model to checkpoints/Optimization1/weights.13.hdf5
632/804 [======================>.......] - ETA: 0s - loss: 0.534

 93/804 [==>...........................] - ETA: 0s - loss: 0.5308 - accuracy: 0.7440
Epoch 00019: saving model to checkpoints/Optimization1/weights.19.hdf5
192/804 [======>.......................] - ETA: 0s - loss: 0.5361 - accuracy: 0.7414
Epoch 00019: saving model to checkpoints/Optimization1/weights.19.hdf5
299/804 [==========>...................] - ETA: 0s - loss: 0.5333 - accuracy: 0.7421
Epoch 00019: saving model to checkpoints/Optimization1/weights.19.hdf5
396/804 [=============>................] - ETA: 0s - loss: 0.5341 - accuracy: 0.7420
Epoch 00019: saving model to checkpoints/Optimization1/weights.19.hdf5
497/804 [=================>............] - ETA: 0s - loss: 0.5331 - accuracy: 0.7424
Epoch 00019: saving model to checkpoints/Optimization1/weights.19.hdf5
598/804 [=====================>........] - ETA: 0s - loss: 0.5340 - accuracy: 0.7414
Epoch 00019: saving model to checkpoints/Optimization1/weights.19.hdf5
706/804 [=========================>....] - ETA: 0s - loss: 0.535

 79/804 [=>............................] - ETA: 0s - loss: 0.5430 - accuracy: 0.7282
Epoch 00025: saving model to checkpoints/Optimization1/weights.25.hdf5
158/804 [====>.........................] - ETA: 0s - loss: 0.5384 - accuracy: 0.7334
Epoch 00025: saving model to checkpoints/Optimization1/weights.25.hdf5
232/804 [=======>......................] - ETA: 0s - loss: 0.5324 - accuracy: 0.7383
Epoch 00025: saving model to checkpoints/Optimization1/weights.25.hdf5
384/804 [=============>................] - ETA: 0s - loss: 0.5360 - accuracy: 0.7388
Epoch 00025: saving model to checkpoints/Optimization1/weights.25.hdf5
455/804 [===============>..............] - ETA: 0s - loss: 0.5364 - accuracy: 0.7389
Epoch 00025: saving model to checkpoints/Optimization1/weights.25.hdf5
580/804 [====================>.........] - ETA: 0s - loss: 0.5363 - accuracy: 0.7382
Epoch 00025: saving model to checkpoints/Optimization1/weights.25.hdf5
652/804 [=======================>......] - ETA: 0s - loss: 0.534

...

  1/804 [..............................] - ETA: 0s - loss: 0.5442 - accuracy: 0.8438
Epoch 00073: saving model to checkpoints/Optimization1/weights.73.hdf5
 68/804 [=>............................] - ETA: 0s - loss: 0.5314 - accuracy: 0.7431
Epoch 00073: saving model to checkpoints/Optimization1/weights.73.hdf5
136/804 [====>.........................] - ETA: 0s - loss: 0.5350 - accuracy: 0.7456
Epoch 00073: saving model to checkpoints/Optimization1/weights.73.hdf5
287/804 [=========>....................] - ETA: 0s - loss: 0.5297 - accuracy: 0.7437
Epoch 00073: saving model to checkpoints/Optimization1/weights.73.hdf5
352/804 [============>.................] - ETA: 0s - loss: 0.5303 - accuracy: 0.7425
Epoch 00073: saving model to checkpoints/Optimization1/weights.73.hdf5
483/804 [=================>............] - ETA: 0s - loss: 0.5288 - accuracy: 0.7448
Epoch 00073: saving model to checkpoints/Optimization1/weights.73.hdf5
553/804 [===================>..........] - ETA: 0s - loss: 0.529

804/804 [==============================] - 1s 773us/step - loss: 0.5331 - accuracy: 0.7422
Epoch 79/100
 63/804 [=>............................] - ETA: 0s - loss: 0.5447 - accuracy: 0.7252
Epoch 00079: saving model to checkpoints/Optimization1/weights.79.hdf5
135/804 [====>.........................] - ETA: 0s - loss: 0.5339 - accuracy: 0.7354
Epoch 00079: saving model to checkpoints/Optimization1/weights.79.hdf5
285/804 [=========>....................] - ETA: 0s - loss: 0.5361 - accuracy: 0.7385
Epoch 00079: saving model to checkpoints/Optimization1/weights.79.hdf5
354/804 [============>.................] - ETA: 0s - loss: 0.5359 - accuracy: 0.7385
Epoch 00079: saving model to checkpoints/Optimization1/weights.79.hdf5
487/804 [=================>............] - ETA: 0s - loss: 0.5330 - accuracy: 0.7408
Epoch 00079: saving model to checkpoints/Optimization1/weights.79.hdf5
556/804 [===================>..........] - ETA: 0s - loss: 0.5337 - accuracy: 0.7404
Epoch 00079: saving model to ch

804/804 [==============================] - 1s 785us/step - loss: 0.5333 - accuracy: 0.7417
Epoch 85/100
 50/804 [>.............................] - ETA: 0s - loss: 0.5440 - accuracy: 0.7381
Epoch 00085: saving model to checkpoints/Optimization1/weights.85.hdf5
109/804 [===>..........................] - ETA: 0s - loss: 0.5352 - accuracy: 0.7382
Epoch 00085: saving model to checkpoints/Optimization1/weights.85.hdf5
232/804 [=======>......................] - ETA: 0s - loss: 0.5322 - accuracy: 0.7431
Epoch 00085: saving model to checkpoints/Optimization1/weights.85.hdf5
302/804 [==========>...................] - ETA: 0s - loss: 0.5313 - accuracy: 0.7436
Epoch 00085: saving model to checkpoints/Optimization1/weights.85.hdf5
441/804 [===============>..............] - ETA: 0s - loss: 0.5341 - accuracy: 0.7411
Epoch 00085: saving model to checkpoints/Optimization1/weights.85.hdf5
515/804 [==================>...........] - ETA: 0s - loss: 0.5325 - accuracy: 0.7418
Epoch 00085: saving model to ch

804/804 [==============================] - 1s 831us/step - loss: 0.5327 - accuracy: 0.7421
Epoch 91/100
  1/804 [..............................] - ETA: 0s - loss: 0.5330 - accuracy: 0.7812
Epoch 00091: saving model to checkpoints/Optimization1/weights.91.hdf5
140/804 [====>.........................] - ETA: 0s - loss: 0.5310 - accuracy: 0.7391
Epoch 00091: saving model to checkpoints/Optimization1/weights.91.hdf5
203/804 [======>.......................] - ETA: 0s - loss: 0.5319 - accuracy: 0.7400
Epoch 00091: saving model to checkpoints/Optimization1/weights.91.hdf5
266/804 [========>.....................] - ETA: 0s - loss: 0.5316 - accuracy: 0.7406
Epoch 00091: saving model to checkpoints/Optimization1/weights.91.hdf5
421/804 [==============>...............] - ETA: 0s - loss: 0.5330 - accuracy: 0.7419
Epoch 00091: saving model to checkpoints/Optimization1/weights.91.hdf5
482/804 [================>.............] - ETA: 0s - loss: 0.5322 - accuracy: 0.7426
Epoch 00091: saving model to ch

804/804 [==============================] - 1s 702us/step - loss: 0.5329 - accuracy: 0.7427
Epoch 97/100
  1/804 [..............................] - ETA: 0s - loss: 0.4607 - accuracy: 0.8438
Epoch 00097: saving model to checkpoints/Optimization1/weights.97.hdf5
 73/804 [=>............................] - ETA: 0s - loss: 0.5111 - accuracy: 0.7641
Epoch 00097: saving model to checkpoints/Optimization1/weights.97.hdf5
142/804 [====>.........................] - ETA: 0s - loss: 0.5262 - accuracy: 0.7524
Epoch 00097: saving model to checkpoints/Optimization1/weights.97.hdf5
299/804 [==========>...................] - ETA: 0s - loss: 0.5294 - accuracy: 0.7471
Epoch 00097: saving model to checkpoints/Optimization1/weights.97.hdf5
366/804 [============>.................] - ETA: 0s - loss: 0.5320 - accuracy: 0.7433
Epoch 00097: saving model to checkpoints/Optimization1/weights.97.hdf5
511/804 [==================>...........] - ETA: 0s - loss: 0.5327 - accuracy: 0.7426
Epoch 00097: saving model to ch

In [20]:

# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5699 - accuracy: 0.7248
Loss: 0.5698808431625366, Accuracy: 0.724781334400177


In [ ]:

# Export our model to HDF5 file
nn.save('AlphabetSoupCharity_Optimization1.h5')